In [1]:
import scanpy as sc
import anndata
import pandas as pd
import pyranges as pr
import pybedtools
import sklearn
import numpy as np


/home/xetrin/miniconda3/envs/rna_atac_env/lib/python3.10/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import scanpy as sc

rna = sc.read_h5ad("/mnt/e/Projects/Brain/SEA_AD_analysis/results/aggregated_multiomic/aggregated_rna_data.h5ad")
print(rna)
sc.pp.filter_cells(rna, min_genes=200)
sc.pp.filter_genes(rna, min_cells=3)
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)
sc.pp.highly_variable_genes(rna, n_top_genes=2000)
rna = rna[:, rna.var.highly_variable]
sc.pp.scale(rna)
sc.tl.pca(rna)
print(rna)


AnnData object with n_obs × n_vars = 138118 × 36601
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mon

/home/xetrin/miniconda3/envs/rna_atac_env/lib/python3.10/site-packages/scanpy/preprocessing/_scale.py:309: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/xetrin/miniconda3/envs/rna_atac_env/lib/python3.10/functools.py:889: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)


AnnData object with n_obs × n_vars = 138118 × 2000
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mont

In [3]:
atac = sc.read_h5ad("/mnt/e/Projects/Brain/SEA_AD_analysis/results/aggregated_multiomic/aggregated_atac_data.h5ad")  # Peak x Cell matrix
print(atac)
sc.pp.normalize_total(atac, target_sum=1e4)
sc.pp.log1p(atac)
print(atac)
# Optional: binarize or apply TF-IDF normalization


AnnData object with n_obs × n_vars = 138118 × 218882
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mo

In [4]:
print(atac.var.head())
peak_coords = atac.var.index.to_series().str.extract(r"(chr[\w]+):(\d+)-(\d+)")
peak_coords.columns = ["Chromosome", "Start", "End"]
peak_coords["Start"] = peak_coords["Start"].astype(int)
peak_coords["End"] = peak_coords["End"].astype(int)
peak_coords["peak_id"] = np.arange(len(peak_coords))

# Create PyRanges object
peaks = pr.PyRanges(peak_coords)
print(peaks)

                                          gene_ids feature_types
chr4:164130572-164131513  chr4:164130572-164131513         Peaks
chr4:16412872-16415576      chr4:16412872-16415576         Peaks
chr4:164116366-164116739  chr4:164116366-164116739         Peaks
chr4:164083047-164083256  chr4:164083047-164083256         Peaks
chr4:164056041-164057013  chr4:164056041-164057013         Peaks
+--------------+-----------+-----------+-----------+
| Chromosome   | Start     | End       | peak_id   |
| (category)   | (int64)   | (int64)   | (int64)   |
|--------------+-----------+-----------+-----------|
| chr1         | 43591097  | 43591599  | 164163    |
| chr1         | 43594973  | 43598008  | 164164    |
| chr1         | 43598680  | 43599017  | 164165    |
| chr1         | 43599915  | 43605694  | 164166    |
| ...          | ...       | ...       | ...       |
| chrY         | 13478174  | 13480771  | 73098     |
| chrY         | 13598891  | 13599575  | 73099     |
| chrY         | 9805339   

In [5]:
genes_df = pd.read_csv("/mnt/e/Projects/Brain/Data/SEAAD/gene_annotations_buffered_5kb.bed", sep="\t", names=["Chromosome", "Start", "End", "Gene"])
genes = pr.PyRanges(genes_df)
print(genes)
peak_to_gene = peaks.nearest(genes)
peak_to_gene = peak_to_gene.df
print(peak_to_gene)
#peak_to_gene = peaks.join(genes)
#print(peak_to_gene)

+--------------+-----------+-----------+------------+
| Chromosome   | Start     | End       | Gene       |
| (category)   | (int64)   | (int64)   | (object)   |
|--------------+-----------+-----------+------------|
| chr1         | 3064168   | 3443621   | PRDM16     |
| chr1         | 5296928   | 5312394   | nan        |
| chr1         | 2398964   | 2418797   | PEX10      |
| chr1         | 5487978   | 5499674   | nan        |
| ...          | ...       | ...       | ...        |
| chrY         | 18686312  | 18729127  | nan        |
| chrY         | 18724882  | 18744197  | TTTY9A     |
| chrY         | 18475247  | 18489237  | nan        |
| chrY         | 18486740  | 18552698  | nan        |
+--------------+-----------+-----------+------------+
Unstranded PyRanges object has 78,649 rows and 4 columns from 24 chromosomes.
For printing, the PyRanges was sorted on Chromosome.
       Chromosome     Start       End  peak_id   Start_b     End_b     Gene  \
0            chr1  43591097  43591

In [6]:
import scipy.sparse as sp

n_peaks = atac.shape[1]
genes = genes_df["Gene"].unique()
gene_to_index = {gene: i for i, gene in enumerate(genes)}

# List of mapping entries
rows = []  # peak indices
cols = []  # gene indices
data = []  # all 1s

for _, row in peak_to_gene.iterrows():
    peak_id = row["peak_id"]
    gene = row["Gene"]
    if pd.notnull(peak_id) and gene in gene_to_index and peak_id < n_peaks:
        rows.append(int(peak_id))
        cols.append(gene_to_index[gene])
        data.append(1)

# peaks × genes sparse matrix
peak_gene_map = sp.csr_matrix((data, (rows, cols)), shape=(n_peaks, len(genes)))

In [ ]:
# ATAC is cells x peaks, peak_gene_map is peaks x genes
access_mat = atac.X @ peak_gene_map  # shape: cells x genes

In [ ]:
peak_counts_per_gene = np.array(peak_gene_map.sum(axis=0)).flatten()  # length = genes

# avoid division by zero
peak_counts_per_gene[peak_counts_per_gene == 0] = 1

access_mat = access_mat.multiply(1 / peak_counts_per_gene)  # element-wise division

In [ ]:
access_mat = access_mat.toarray()

In [ ]:
atac_gene = anndata.AnnData(X=access_mat)
atac_gene.obs_names = atac.obs_names
atac_gene.var_names = genes

In [ ]:
print(atac_gene)

In [ ]:
print(atac_gene.X[1:5,1:5])

In [ ]:
from sklearn.cross_decomposition import CCA

# Reduce dimensionality
sc.pp.pca(rna, n_comps=30)
sc.pp.pca(atac_gene, n_comps=30)

cca = CCA(n_components=20)
rna_cca, atac_cca = cca.fit_transform(rna.obsm["X_pca"], atac_gene.obsm["X_pca"])

In [ ]:
shared_embedding = np.concatenate([rna_cca, atac_cca])

In [ ]:
# Combine
joint = anndata.AnnData(X=shared_embedding)
joint.obs["modality"] = ["rna"] * rna.n_obs + ["atac"] * atac.n_obs

sc.pp.neighbors(joint, n_neighbors=15)
sc.tl.umap(joint)
sc.tl.leiden(joint)
sc.pl.umap(joint, color=["modality", "leiden"])

In [ ]:
joint.obs["Subclass"] = (
    list(rna.obs["Subclass"] if "Subclass" in rna.obs else ["NA"] * rna.n_obs)
    + list(atac.obs["Subclass"])
)

In [ ]:
print(joint.obs.columns)

In [ ]:
sc.pl.umap(joint, color=["Subclass"])